In [2]:
from keras.models import load_model
model = load_model("modelo_completo.keras") 

2025-10-17 00:21:46.514571: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import backtrader as bt
import numpy as np
import yfinance as yf
import pandas as pd
from keras.models import load_model

from datetime import datetime

start = "2018-01-01"
end = "2025-06-01"
start_date = datetime.strptime(start, "%Y-%m-%d")
end_date = datetime.strptime(end, "%Y-%m-%d")
diff_days = (end_date - start_date).days

totalYears = diff_days / 365.25
initialCapital = 10000.0


class KerasCNNStrategy(bt.Strategy):
    params = (
        ('window_size', 60),
        ('model_path', 'modelo_completo.keras'),
    )

    def __init__(self):
        self.model = load_model(self.p.model_path)

        # Indicators on the open price
        self.ema9 = bt.ind.EMA(self.data.open, period=9)
        self.sma20 = bt.ind.SMA(self.data.open, period=20)
        self.sma200 = bt.ind.SMA(self.data.open, period=200)

        self.data_buffer = []

    def next(self):
        # Append data for current bar
        self.data_buffer.append(self._get_indicator_values())

        if len(self.data_buffer) < self.p.window_size:
            return

        window_arr = np.array(self.data_buffer[-self.p.window_size:])
        X_input = np.expand_dims(window_arr, axis=0)

        prob = float(self.model.predict(X_input, verbose=0)[0][0])

        open_price = float(self.data.open[0])
        close_price = float(self.data.close[0])

        # Define the trade size (por exemplo, 10 ações)
        trade_size = int(self.broker.getcash() / open_price * 0.01)  # usa 1% do capital
  

        if not self.position:
            if prob > 0.5:
                self.buy(size=trade_size)
                # print(f"BUY  {trade_size} shares at {open_price:.2f} | Prob: {prob:.3f}")
            elif prob < 0.5:
                self.sell(size=trade_size)
                # print(f"SELL {trade_size} shares at {open_price:.2f} | Prob: {prob:.3f}")
        else:
            # print(f"CLOSE position at {close_price:.2f}")
            self.close()


    def _get_indicator_values(self):
        return [
            float(self.data.open[0]),
            float(self.ema9[0]),
            float(self.sma20[0]),
            float(self.sma200[0])
        ]

def run_backtest():
    cerebro = bt.Cerebro()
    cerebro.addstrategy(KerasCNNStrategy)

    # Download data
    df = yf.download("MRK", start=start, end=end, group_by='ticker')
    df.columns = df.columns.droplevel(0)  # Drops the 'MRK' level
    # Flatten multi-index columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df.index = pd.to_datetime(df.index)

    # Feed data to Backtrader
    feed = bt.feeds.PandasData(
        dataname=df,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        datetime=None
    )

    cerebro.adddata(feed)
    cerebro.broker.setcash(initialCapital)
    cerebro.broker.setcommission(commission=0.005)

    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
    cerebro.run()
    percentualProfit = finalCapital / initialCapital
    percentualProfitPerAnualized = percentualProfit / totalYears
    print(f"Total Years: {totalYears:.2f}")
    print(f"Final Portfolio Value: {finalCapital:.2f}")
    print(f"Total Profit (x times): {percentualProfit:.2f}")
    print(f"Annualized Profit (x times per year): {percentualProfitPerAnualized:.2f}")
    print(f"Annualized Profit (% per year): {(percentualProfitPerAnualized - 1) * 100:.2f}%")
    cerebro.plot()


if __name__ == "__main__":
    run_backtest()


/var/folders/11/l_xl0mcx11vb3dr5w91sdcbh0000gn/T/ipykernel_7339/2599283562.py:79: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("MRK", start=start, end=end, group_by='ticker')
[*********************100%***********************]  1 of 1 completed


Starting Portfolio Value: 10000.00
SELL 1 shares at 57.87 | Prob: 0.491
CLOSE position at 56.84
SELL 1 shares at 56.63 | Prob: 0.491
CLOSE position at 57.80
SELL 1 shares at 57.57 | Prob: 0.491
CLOSE position at 58.78
SELL 1 shares at 58.55 | Prob: 0.491
CLOSE position at 58.45
SELL 1 shares at 58.23 | Prob: 0.491
CLOSE position at 56.52
SELL 1 shares at 56.17 | Prob: 0.491
CLOSE position at 56.73
SELL 1 shares at 56.91 | Prob: 0.491
CLOSE position at 57.66
SELL 1 shares at 58.86 | Prob: 0.491
CLOSE position at 59.55
SELL 1 shares at 59.95 | Prob: 0.491
CLOSE position at 59.96
SELL 1 shares at 59.61 | Prob: 0.491
CLOSE position at 60.06
SELL 1 shares at 60.27 | Prob: 0.491
CLOSE position at 60.83
SELL 1 shares at 60.86 | Prob: 0.491
CLOSE position at 61.16
SELL 1 shares at 61.71 | Prob: 0.491
CLOSE position at 61.39
SELL 1 shares at 61.20 | Prob: 0.491
CLOSE position at 61.85
SELL 1 shares at 61.80 | Prob: 0.491
CLOSE position at 62.27
SELL 1 shares at 62.38 | Prob: 0.491
CLOSE positio

NameError: name 'finalCapital' is not defined